# Assignment

## Importing Libraries

In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import seaborn as sns

#!pip install torch-directml
import torch_directml
dml = torch_directml.device()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f'Currently using {device}')

## Loading the Datasets

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

cifar_dataset_train = datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)

cifar_dataset_test = datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
classes = ()

## Creating the training dataset

In [4]:
from torch.utils.data import DataLoader
batch_size = 100

train_iter = DataLoader(cifar_dataset_train,batch_size,shuffle=True,num_workers=2)

In [5]:
X,y = next(iter(train_iter))

## Creating the test dataset

In [6]:
test_iter = DataLoader(cifar_dataset_test,batch_size,shuffle=False,num_workers=2)

In [7]:
X_test,y_test = next(iter(test_iter))

## Analysing the CIFAR-10 data

In [8]:
print(f'The dimensions of each input image is {X.size()} The number of input channels is {X.size()[1]}')
print(f'The output labels are as follows {y}')

The dimensions of each input image is torch.Size([100, 3, 32, 32]) The number of input channels is 3
The output labels are as follows tensor([6, 1, 3, 9, 0, 6, 5, 2, 5, 3, 8, 3, 0, 3, 8, 2, 4, 2, 9, 5, 4, 5, 7, 3,
        0, 3, 6, 3, 2, 6, 6, 4, 1, 0, 7, 9, 0, 1, 5, 2, 9, 1, 3, 0, 9, 7, 1, 0,
        9, 1, 8, 4, 4, 5, 0, 4, 1, 5, 4, 0, 1, 4, 1, 0, 2, 7, 0, 0, 7, 0, 4, 3,
        1, 0, 6, 0, 1, 5, 0, 1, 1, 3, 7, 4, 3, 7, 2, 4, 2, 6, 1, 1, 6, 4, 4, 5,
        9, 8, 6, 0])


## Example image

## CNN Architecture

In [70]:
class Block(torch.nn.Module):
    def __init__(self,input_channels,output_channels,strides,features,out_features,use_1x1conv=False):
        super(Block,self).__init__()
        self.output_channels = output_channels

        self.b1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=3,stride=strides,padding=1),
            torch.nn.BatchNorm2d(output_channels),
            torch.nn.ReLU(),
        )

        self.b2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=5,stride=strides,padding=2),
            torch.nn.BatchNorm2d(output_channels),
            torch.nn.ReLU(),
        )

        self.linear = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d((8,8)),
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=features,out_features=out_features),
            torch.nn.ReLU(),
            torch.nn.Dropout(0)
        )

        if use_1x1conv:
            self.conv = torch.nn.Conv2d(input_channels,output_channels,kernel_size=1,stride=strides)

        else:
            self.conv = None

    def forward(self,x):

        b1 = self.b1(x)
        b2 = self.b2(x)

        linear = self.linear(x)

        o1 = (linear[:,0].view(100,-1) * b1.view(100,-1))
        o2 = (linear[:,1].view(100,-1) * b2.view(100,-1))

        out = (o1+o2).view(100,self.output_channels,x.size(2),-1)

        if self.conv:
            x = self.conv(x)

        out = torch.nn.functional.relu(x + out)

        return out

In [71]:
class CNN(torch.nn.Module):
    def __init__(self,num_classes):
        super(CNN,self).__init__()

        self.stem = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,out_channels=3,kernel_size=1,stride=1,padding=0),
            torch.nn.BatchNorm2d(3),
            torch.nn.ReLU(),
        )

        self.block1 = Block(input_channels=3,output_channels=64,features=192,out_features=2,strides=1,use_1x1conv=True)
        self.block2 = Block(input_channels=64,output_channels=128,features=4096,out_features=2,strides=1,use_1x1conv=True)

        self.last = torch.nn.Sequential(torch.nn.AdaptiveAvgPool2d((8,8)),
                                        torch.nn.Flatten(),
                                        torch.nn.Linear(in_features=8192,out_features=20),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(0)
                                        )

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=20,out_features=num_classes),
            torch.nn.ReLU(),
            torch.nn.Dropout(0),
            torch.nn.Softmax(dim=1)
        )

    def forward(self,x):

        output = self.stem(x)

        output = self.block1(output)
        output = self.block2(output)

        output = self.last(output)

        output = self.linear(output)

        return output

In [72]:
model = CNN(num_classes=10).to(dml)
print(model)

CNN(
  (stem): Sequential(
    (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block1): Block(
    (b1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (b2): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(6, 6))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (linear): Sequential(
      (0): AdaptiveAvgPool2d(output_size=(8, 8))
      (1): Flatten(start_dim=1, end_dim=-1)
      (2): Linear(in_features=192, out_features=2, bias=True)
      (3): ReLU()
      (4): Dropout(p=0, inplace=False)
    )
    (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (block2): Block(
    (b1): Sequential(
      (0): Co

In [73]:
model(X.to(dml)).size()

RuntimeError: The size of tensor a (65536) must match the size of tensor b (28224) at non-singleton dimension 1

## Create Loss and Optimization

In [51]:
loss = torch.nn.CrossEntropyLoss()

# Create the optimizer term
optimizer = torch.optim.SGD(model.parameters(), lr=0.005,weight_decay=0.005,momentum=0.5)

## Calculate Loss, and Validation Accuracy

In [52]:
def train_test_model(model,train_iter,test_iter,epochs,optimizer,loss):
    for epoch in range(epochs): # Initialize the epochs

        l_score = []

        for i,(X,y) in enumerate(train_iter):

            X = X.to(dml)
            y = y.to(dml)

            optimizer.zero_grad()

            #model.train()
            y_hat = model(X)
            l = loss(y_hat,y)
            l.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {l.item():.4f}')

        with torch.no_grad():
            correct = 0
            total = 0
            for data in test_iter:
                X_test,y_test = data
                X_test = X_test.to(dml)
                y_test = y_test.to(dml)

                y_hat = model(X_test)

                _, predicted = torch.max(y_hat.data,1)
                total += y_test.size(0)
                correct += (predicted == y_test).sum().item()

            print(f'Accuracy of the network on the test images is : {100 * correct // total} percent')

    return

In [53]:
epochs = 10
train_test_model(model,train_iter,test_iter,epochs,optimizer,loss)

Epoch [1/10], Loss: 2.1421
Accuracy of the network on the test images is : 28 percent
Epoch [2/10], Loss: 2.1807
Accuracy of the network on the test images is : 32 percent
Epoch [3/10], Loss: 2.1016
Accuracy of the network on the test images is : 33 percent
Epoch [4/10], Loss: 2.0853
Accuracy of the network on the test images is : 35 percent
Epoch [5/10], Loss: 2.0795
Accuracy of the network on the test images is : 38 percent
Epoch [6/10], Loss: 2.0220
Accuracy of the network on the test images is : 39 percent
Epoch [7/10], Loss: 1.9938
Accuracy of the network on the test images is : 42 percent
Epoch [8/10], Loss: 2.0255
Accuracy of the network on the test images is : 43 percent
Epoch [9/10], Loss: 1.9817
Accuracy of the network on the test images is : 43 percent
Epoch [10/10], Loss: 2.0374
Accuracy of the network on the test images is : 44 percent
